### Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
import hypernetx as hnx
from hypernetx.utils import toys

import warnings
import importlib as imp
from collections import defaultdict
from copy import deepcopy
warnings.simplefilter('ignore')

In [2]:
imp.reload(hnx)

<module 'hypernetx' from '/Users/prag717/Library/CloudStorage/OneDrive-PNNL/Documents/tdm/hnx/hypernetx/__init__.py'>

## Props2DFP

In [9]:
### test props
import ast,json
from hypernetx import HyperNetXError

def mkdict(x):
    if isinstance(x,dict):
        return x
    else:
        try:
            temp = json.loads(x)
        except:
            try:
                temp = literal_eval(x)
            except:
                temp = {}
        if isinstance(temp,dict):
            return temp
        else:
            return {}
            # raise HyperNetXError('input not convertible to a dictionary')
    
def props2dfp(properties,
                weight_prop,
                default_weight,
                misc_properties_col=None,):
    """
    Changes properties input into standard dfp format: id,weight,...<other shared properties>...,misc properties dict
    Used for single level (edges or nodes or combined) 

    Parameters
    ----------
    properties: dict | pd.Dataframe
        if dict, must be of the form {id:properties, ...}
        if dataframe, index must be single identifier (id)
    weight_prop: Optional[str] = None
    default_weight: int|float|Sequence[int|float] = 1
    misc_properties_col: Optional[str] = None

    Returns
    -------
    pd.Dataframe

    """
    if isinstance(properties, dict):
        dp = deepcopy(properties)
        dfp = pd.DataFrame(pd.Series(dp))
        dfp = dfp.reset_index().rename(columns={"index":'uid',0:'properties'}) 
        dfp['weight'] = default_weight 
        prop_flag = 1

    elif isinstance(properties, pd.DataFrame):
        dfp = deepcopy(properties)
        if weight_prop is not None and weight_prop in dfp.columns:
            dfp = dfp.rename(columns={weight_prop: 'weight'})
            dfp = dfp.fillna(
                {'weight': default_weight}
            ) 
        else:
            dfp['weight'] = default_weight 
        if  misc_properties_col is not None:
            dfp = dfp.rename(columns={misc_properties_col: 'properties'}) 
            prop_flag = 1
            dfp.properties.fillna({})
        else:
            dfp['properties'] = [{} for row in dfp]
            prop_flag = 0
    else:
        raise ValueError('properties must be a dictionary or a Pandas dataframe')
    
    cols = [c for c in dfp.columns if c not in ['uid','weight','properties'] ] 
    dfp = dfp[['uid','weight'] + cols + ['properties']] 
    
    if prop_flag == 0:
        return dfp
    else:
        props = dfp.properties
        # raise
        dfp.properties = dfp.properties.map(mkdict)

        for idx,row in enumerate(dfp.itertuples()):
            wt = row.properties.get(weight_prop,None)

            if wt is not None:
                dfp.weight.iloc[idx] = wt
                del dfp.properties.iloc[idx][weight_prop]
        return dfp

In [10]:
h = hnx.Hypergraph()

/Users/prag717/Library/CloudStorage/OneDrive-PNNL/Documents/tdm/hnx/hypernetx/classes/hypergraph.py:334: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1]) ##Nodes(self.E) ##


In [11]:
h._state_dict

{'dataframe': Empty DataFrame
 Columns: [edges, nodes, cell_weights]
 Index: [],
 'labels': {'edges': array([], dtype=float64),
  'nodes': array([], dtype=float64)},
 'data': array([], shape=(2, 0), dtype=float64),
 'snodelg': {},
 'sedgelg': {},
 'neighbors': defaultdict(dict, {}),
 'edge_neighbors': defaultdict(dict, {}),
 'adjacency_matrix': {},
 'edge_adjacency_matrix': {}}

In [12]:
h.E._dataframe

,edges,nodes,cell_weights


In [13]:
h.nodes.properties

,,uid,weight,properties
level,id,,,


In [14]:
E = h.E.restrict_to_levels(1)

/var/folders/93/nv43byrs4jv_qsn68r4fdjmr0000gn/T/ipykernel_40706/2351284712.py:1: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  E = h.E.restrict_to_levels(1)


In [15]:
E.dataframe

,edges,nodes,cell_weights


## Dataframes

### Dogs and Cats

In [16]:
dogs =  pd.read_excel('dogsAndcats.xls',sheet_name='dogs').set_index('uid')
cats = pd.read_excel('dogsAndcats.xls',sheet_name='cats')
pairs = pd.read_excel('dogsAndcats.xls',sheet_name='pairs')


dogs

,breed,weight,misc_properties
uid,,,
Togo,husky,90,"{""weight"":92,""color"":""white""}"
Balto,husky,70,NaN
Willow,sibercaan,90,"{""died"":2020}"
Tux,ausie,50,"{""died"":2006}"
Buddy,lab,80,"{""died"":2001}"
Bandit,golden,80,"{""died"":2003}"


In [17]:
cats = pd.read_csv('cats_alt.csv').set_index('id').to_dict()
cats = list(cats.values())[0]
dfp = props2dfp(cats,'weight',20)
weight_prop = "weight"
default_weight = 20
misc_properties_col = None
# dfp = props2dfp(cats,"weight",
#                 20,
#                 )
props = dfp.properties
# def mkdict(x):
#     try:
#         temp = ast.literal_eval(x)
#         if not isinstance(temp,dict):
#             temp = {}
#     except:
#         temp = {}
#     return temp

dfp.properties = dfp.properties.map(mkdict)

for idx,row in enumerate(dfp.itertuples()):
    wt = row.properties.get(weight_prop,None)

    if wt is not None:
        dfp.weight.iloc[idx] = wt
        del dfp.properties.iloc[idx][weight_prop]
dfp
        
# for idx in props.index :
#     if not isinstance(props.loc[idx],dict):
#         try:
#             temp = ast.literal_eval(props.iloc[idx])

#             print(18,temp)
#         except:
#             temp = {}
#             print(21,temp)
#     wt = temp.get(weight_prop,None)
#     if wt is not None:
#         dfp.iloc[idx].weight = wt
#         print(25,dfp)
# #         del temp[weight_prop]
#         print(27,temp)
#     dfp.properties.iloc[idx] = temp
#     print(29,dfp)

# wt = ast.literal_eval(dfp.properties.loc[1]).get('weight',None)
# wt

,uid,weight,properties
0,Amber,20,{}
1,Bill,20,{}
2,Cozy,20,{}
3,Daisy,20,{}


In [18]:
dogs =  pd.read_excel('dogsAndcats.xls',sheet_name='dogs')
# print(dogs)
# print(type(dogs.misc_properties.loc[0]))
props2dfp(dogs,"weight",
                100,
                "misc_properties")

,uid,weight,breed,properties
0,Togo,92,husky,{'color': 'white'}
1,Balto,70,husky,{}
2,Willow,90,sibercaan,{'died': 2020}
3,Tux,50,ausie,{'died': 2006}
4,Buddy,80,lab,{'died': 2001}
5,Bandit,80,golden,{'died': 2003}


In [20]:
dogs.uid = dogs.uid.astype('category')
dogs.uid

0      Togo
1     Balto
2    Willow
3       Tux
4     Buddy
5    Bandit
Name: uid, dtype: category
Categories (6, object): ['Balto', 'Bandit', 'Buddy', 'Togo', 'Tux', 'Willow']

In [9]:
dogs

,id,breed,weight,misc_properties
0,Togo,husky,90,"{""weight"":92,""color"":""white""}"
1,Balto,husky,70,NaN
2,Willow,sibercaan,90,"{""died"":2020}"
3,Tux,ausie,50,"{""died"":2006}"
4,Buddy,lab,80,"{""died"":2001}"
5,Bandit,golden,80,"{""died"":2003}"


In [27]:
cats =  pd.read_excel('dogsAndcats.xls',sheet_name='cats').set_index('uid').to_dict()
cats = list(cats.values())[0]
print(cats)
print(type(cats["Amber"]))
cats = props2dfp(cats,"weight",25)
cats

{'Amber': "{'breed':'Persian'}", 'Bill': "{'weight':10,'name':'Tiger'}", 'Cozy': "{'weight':35}", 'Daisy': "{'breed':'Siamese'}", 'Fred': '{}', 'George': '{}', 'Henry': '{}'}
<class 'str'>


,id,weight,properties
0,Amber,25,{}
1,Bill,25,{}
2,Cozy,25,{}
3,Daisy,25,{}
4,Fred,25,{}
5,George,25,{}
6,Henry,25,{}


In [11]:
cats

,id,weight,properties
0,Amber,25,{}
1,Bill,25,{}
2,Cozy,25,{}
3,Daisy,25,{}
4,Fred,25,{}
5,George,25,{}
6,Henry,25,{}


In [12]:
jsoncats = {k:json.dumps(mkdict(v)) for k,v in cats.items()}

props2dfp(jsoncats,'weight',45)

,id,weight,properties
0,id,45,{}
1,weight,45,{}
2,properties,45,{}


In [13]:
dogs_cats

NameError: name 'dogs_cats' is not defined

In [14]:

# dogs = dogs.set_index('id')
# cats = cats.set_index('id')
cats

,id,weight,properties
0,Amber,25,{}
1,Bill,25,{}
2,Cozy,25,{}
3,Daisy,25,{}
4,Fred,25,{}
5,George,25,{}
6,Henry,25,{}


In [45]:
dogs_cats =  pd.read_excel('dogsAndcats.xls',sheet_name='pairs')
dogs_cats.edges = dogs_cats.edges.astype('category')
dogs_cats.nodes = dogs_cats.nodes.astype('category')
dogs_cats = dogs_cats.set_index(['edges','nodes'])

dogs_cats.index

MultiIndex([(  'Togo',  'Amber'),
            (  'Togo', 'George'),
            ( 'Balto',   'Bill'),
            ( 'Balto',   'Fred'),
            (   'Tux',  'Daisy'),
            (   'Tux',   'Fred'),
            (   'Tux',  'Henry'),
            ( 'Buddy',   'Bill'),
            ('Bandit',   'Cozy'),
            ('Bandit',  'Amber'),
            ('Bandit',   'Fred'),
            ('Bandit',   'Bill')],
           names=['edges', 'nodes'])

In [102]:
dogs.index,cats.index,dogs_cats.index

(CategoricalIndex(['Togo', 'Balto', 'Willow', 'Tux', 'Buddy', 'Bandit'], categories=['Balto', 'Bandit', 'Buddy', 'Togo', 'Tux', 'Willow'], ordered=False, dtype='category', name='id'),
 CategoricalIndex(['Amber', 'Bill', 'Cozy', 'Daisy', 'Fred', 'George', 'Henry'], categories=['Amber', 'Bill', 'Cozy', 'Daisy', 'Fred', 'George', 'Henry'], ordered=False, dtype='category', name='id'),
 MultiIndex([(  'Togo',  'Amber'),
             (  'Togo', 'George'),
             ( 'Balto',   'Bill'),
             ( 'Balto',   'Fred'),
             (   'Tux',  'Daisy'),
             (   'Tux',   'Fred'),
             (   'Tux',  'Henry'),
             ( 'Buddy',   'Bill'),
             ('Bandit',   'Cozy'),
             ('Bandit',  'Amber'),
             ('Bandit',   'Fred'),
             ('Bandit',   'Bill')],
            names=['edge', 'node']))